### **I'm attempting to create my own progressive dataset based on the season's progression to capture new features not captured in nba_api that might help to improve accuracy. By the end of the season, I will once again check accuracy. The new feature of interest this season is injuries going into each game.**
### **A "half" injury is a day-to-day/game-time decision**

In [1]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 14.1 MB/s eta 0:00:00


In [3]:
import nba_api
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd
import numpy as np

gl = leaguegamelog.LeagueGameLog(season='2025-26', season_type_all_star='Regular Season')
df = gl.get_data_frames()[0]
df['HOME_OR_AWAY'] = np.where(
    df['MATCHUP'].str.contains(' vs. '), 1,
    np.where(df['MATCHUP'].str.contains(' @ '), 0, np.nan)
)
df['MATCHUP'] = df['MATCHUP'].apply(lambda x: x.split(' ')[2])
df['STARTER_INJURIES'] = [0, 1.5, 1, 1, 0, 1.5, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0.5, 0.5, 1.5, 0, 0, 1, 0, 1,
                          1, 0.5, 1, 0, 1.5, 0, 1, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0.5, 0, 0, 1, 1, 0.5, 0, 1, 1,
                          1, 1.5, 0, 1, 2, 0, 1, 1, 0, 1, 1.5, 1, 2, 1, 0, 1, 2, 1.5, 0, 1.5, 0, 0, 0, 1, 0.5,
                          0.5, 0.5, 0, 1, 1.5, 1, 1, 1, 0.5, 0, 1, 0, 1, 1.5, 0, 2, 1.5, 1, 0, 1.5, 0, 0, 1.5, 0, 0.5,
                          0, 0, 1, 1.5, 0, 1, 1, 2, 1, 1, 0, 1, 1.5, 1, 1, 1, 0, 0.5, 0, 0.5, 2, 0.5, 0, 1, 0,
                          1, 1, 0, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 0, 0, 2, 1.5, 0, 1.5, 0, 1, 1, 0, 1, 1, 2, 1.5,
                          1, 2, 0, 2, 0, 1, 1, 0.5, 2, 1, 0, 0, 3, 0, 0, 0, 0, 2, 1, 1.5, 1.5, 0, 2, 2, 1,
                          1, 1.5, 1.5, 1, 2, 1, 0, 0, 1, 2, 0.5, 2, 1, 0.5, 2, 0.5, 0, 0.5, 1, 2, 1, 2, 0, 2, 0.5,
                          2, 2, 0, 0.5, 0, 1, 1.5, 1, 1, 1, 1, 0, 0, 1, 1, 1.5, 2.5, 0, 1.5, 0, 2, 1.5, 1, 1, 0.5,
                          1, 0.5, 2, 2, 0, 1, 1, 1.5, 1, 1.5, 1.5, 1, 1, 0.5, 0.5, 2, 2, 0, 0, 1, 1, 2, 1, 0, 0.5,
                          1, 1.5, 0, 1, 0, 0, 0, 0.5, 2, 0.5, 1.5, 1, 1, 1, 0, 1, 2, 2, 1.5, 0, 2, 0, 2, 0.5, 1,
                          2, 1, 1, 1, 1, 0.5, 0, 0, 1.5, 1, 0, 0, 0, 1, 0.5, 0.5, 1, 1, 0, 1, 0, 1, 1.5, 1, 2,
                          0.5, 1, 0, 1, 1.5, 1, 2, 2, 0, 0, 1, 0.5, 0, 0, 1, 0.5, 0, 0.5, 1, 1, 1, 0, 2, 0, 3,
                          1, 1, 0, 0, 1.5, 1, 1, 0, 0, 2, 0, 1, 3, 2, 0.5, 1.5, 0, 2, 1.5, 2, 0, 0.5, 0, 3, 0,
                          0, 0, 1, 0, 1, 2.5, 3, 0, 1, 0, 2, 2, 0.5, 0, 2, 2.5, 0, 0.5, 1, 0, 1, 0, 1, 1, 1, 2,
                          1, 1, 0, 0, 0, 1.5, 0.5, 1, 0, 1, 0, 0, 1, 0, 1.5, 0, 1, 1, 2, 0, 1.5, 1]

df['W/L'] = np.where(df['WL'] == 'W', 1, 0)
df = df.drop(columns=['SEASON_ID', 'TEAM_NAME', 'WL', 'VIDEO_AVAILABLE', 'PLUS_MINUS'])
df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG_PCT,FG3M,...,REB,AST,STL,BLK,TOV,PF,PTS,HOME_OR_AWAY,STARTER_INJURIES,W/L
0,1610612745,HOU,0022500001,2025-10-21,OKC,290,43,97,0.443,11,...,52,23,6,5,25,26,124,0.0,0.0,0
1,1610612744,GSW,0022500002,2025-10-21,LAL,240,38,78,0.487,17,...,40,29,10,4,19,27,119,0.0,1.5,1
2,1610612747,LAL,0022500002,2025-10-21,GSW,240,42,77,0.545,8,...,39,23,7,2,20,21,109,1.0,1.0,0
3,1610612760,OKC,0022500001,2025-10-21,HOU,290,46,104,0.442,13,...,38,29,12,4,12,27,125,1.0,1.0,1
4,1610612737,ATL,0022500082,2025-10-22,TOR,240,38,90,0.422,10,...,34,25,7,6,16,24,118,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1610612737,ATL,0022500239,2025-11-16,PHX,240,45,85,0.529,14,...,34,35,11,3,17,19,124,0.0,1.0,1
394,1610612746,LAC,0022500233,2025-11-16,BOS,240,38,81,0.469,14,...,44,23,4,7,14,20,118,0.0,2.0,0
395,1610612738,BOS,0022500233,2025-11-16,LAC,240,46,103,0.447,16,...,49,26,6,6,6,21,121,1.0,0.0,1
396,1610612741,CHI,0022500240,2025-11-16,UTA,290,49,108,0.454,19,...,60,38,7,10,17,29,147,0.0,1.5,0


In [12]:
matchups_df = df.merge(
    df,
    left_on=['GAME_ID', 'TEAM_ABBREVIATION', 'MATCHUP'],
    right_on=['GAME_ID', 'MATCHUP', 'TEAM_ABBREVIATION'],
    suffixes=('', '_OPP')
)
matchups_df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG_PCT,FG3M,...,REB_OPP,AST_OPP,STL_OPP,BLK_OPP,TOV_OPP,PF_OPP,PTS_OPP,HOME_OR_AWAY_OPP,STARTER_INJURIES_OPP,W/L_OPP
0,1610612745,HOU,0022500001,2025-10-21,OKC,290,43,97,0.443,11,...,38,29,12,4,12,27,125,1.0,1.0,1
1,1610612744,GSW,0022500002,2025-10-21,LAL,240,38,78,0.487,17,...,39,23,7,2,20,21,109,1.0,1.0,0
2,1610612747,LAL,0022500002,2025-10-21,GSW,240,42,77,0.545,8,...,40,29,10,4,19,27,119,0.0,1.5,1
3,1610612760,OKC,0022500001,2025-10-21,HOU,290,46,104,0.442,13,...,52,23,6,5,25,26,124,0.0,0.0,0
4,1610612737,ATL,0022500082,2025-10-22,TOR,240,38,90,0.422,10,...,54,36,10,4,19,31,138,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1610612737,ATL,0022500239,2025-11-16,PHX,240,45,85,0.529,14,...,47,23,9,3,17,24,122,1.0,1.0,0
394,1610612746,LAC,0022500233,2025-11-16,BOS,240,38,81,0.469,14,...,49,26,6,6,6,21,121,1.0,0.0,1
395,1610612738,BOS,0022500233,2025-11-16,LAC,240,46,103,0.447,16,...,44,23,4,7,14,20,118,0.0,2.0,0
396,1610612741,CHI,0022500240,2025-11-16,UTA,290,49,108,0.454,19,...,53,29,13,6,9,28,150,1.0,1.0,1


In [15]:
X = matchups[['TEAM_ID', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'PTS', 'HOME_OR_AWAY', 'STARTER_INJURIES', 'TEAM_ID_OPP', 'FGM_OPP', 'FGA_OPP', 'FG_PCT_OPP', 'FG3M_OPP', 'FG3A_OPP', 'FG3_PCT_OPP',
        'FTM_OPP', 'FTA_OPP', 'FT_PCT_OPP', 'OREB_OPP', 'DREB_OPP', 'REB_OPP', 'AST_OPP', 'TOV_OPP', 'STL_OPP', 'BLK_OPP', 'PF_OPP', 'PTS_OPP', 'HOME_OR_AWAY_OPP', 'STARTER_INJURIES_OPP']]
y_wl = matchups[['W/L']]

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

X_train, X_test, y_train, y_test = train_test_split(X, y_wl.values.ravel(), test_size=0.25, random_state=42)

clf = LogisticRegression(max_iter=1000)
beta = clf.fit(X_train, y_train)

y_pred = beta.predict(X_test)
y_pred_proba = beta.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)

print(loss)
print("Accuracy:", accuracy_score(y_test, y_pred))

0.6937742197073304
Accuracy: 0.48


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE

results = []

for k in range(1, X.shape[1]+1):
    rfe = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=k)
    X_rfe = rfe.fit_transform(X_train, y_train)
    score = cross_val_score(LogisticRegression(max_iter=1000), X_rfe, y_train, cv=5).mean()
    results.append((k, score))

results_df = pd.DataFrame(results, columns=['Num_Features', 'CV_Accuracy'])
print(results_df.sort_values('CV_Accuracy', ascending=False))

    Num_Features  CV_Accuracy
0              1     0.506723
1              2     0.506723
2              3     0.506723
3              4     0.506723
4              5     0.506723
5              6     0.506723
6              7     0.506723
7              8     0.506723
8              9     0.506723
9             10     0.506723
10            11     0.506723
11            12     0.506723
12            13     0.506723
13            14     0.506723
14            15     0.506723
15            16     0.506723
16            17     0.506723
17            18     0.506723
18            19     0.506723
19            20     0.506723
20            21     0.506723
21            22     0.506723
22            23     0.506723
23            24     0.506723
24            25     0.506723
25            26     0.506723
26            27     0.506723
27            28     0.506723
28            29     0.506723
29            30     0.506723
30            31     0.506723
31            32     0.506723
32        

In [18]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': clf.coef_[0]
}).sort_values(by='Coefficient', key=abs, ascending=False)

print(coef_df.head(31))

        Feature   Coefficient
21  TEAM_ID_OPP  8.334484e-12
0       TEAM_ID  8.334483e-12
18          PTS  8.411023e-18
39      PTS_OPP -6.229937e-18
1           FGM  3.146798e-18
12          REB  2.671761e-18
13          AST  2.495779e-18
11         DREB  2.475059e-18
22      FGM_OPP -2.279615e-18
32     DREB_OPP -2.072128e-18
33      REB_OPP -1.885923e-18
34      AST_OPP -1.749938e-18
4          FG3M  1.157437e-18
2           FGA  1.085996e-18
28      FTM_OPP -1.001520e-18
7           FTM  9.599890e-19
35      TOV_OPP  8.970644e-19
15          STL  8.621176e-19
29      FTA_OPP -8.482649e-19
8           FTA  6.996408e-19
25     FG3M_OPP -6.691877e-19
36      STL_OPP -5.920361e-19
38       PF_OPP  5.885504e-19
14          TOV -5.329676e-19
16          BLK  5.167306e-19
17           PF -5.105333e-19
37      BLK_OPP -4.111565e-19
26     FG3A_OPP  3.849770e-19
23      FGA_OPP  3.487904e-19
5          FG3A  3.038765e-19
10         OREB  1.967026e-19


In [19]:
X_verify = matchups[['TEAM_ID', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'PTS', 'TEAM_ID_OPP', 'FGM_OPP', 'FGA_OPP', 'FG3M_OPP', 'FG3A_OPP',
        'FTM_OPP', 'FTA_OPP', 'DREB_OPP', 'REB_OPP', 'AST_OPP', 'TOV_OPP', 'STL_OPP', 'BLK_OPP', 'PF_OPP', 'PTS_OPP']]

X_train, X_test, y_train, y_test = train_test_split(X_verify, y_wl.values.ravel(), test_size=0.25, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)

print(loss)
print("Accuracy:", accuracy_score(y_test, y_pred))

0.6937742197073304
Accuracy: 0.48


In [20]:
from nba_api.stats.endpoints import leaguedashteamstats

team_stats = leaguedashteamstats.LeagueDashTeamStats(season='2025-26', league_id_nullable='00').get_data_frames()[0]
team_stats

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,1610612737,Atlanta Hawks,14,9,5,0.643,672.0,596,1227,0.486,...,15,1,20,3,6,4,14,12,6,12
1,1610612738,Boston Celtics,14,7,7,0.500,672.0,592,1319,0.449,...,6,20,1,12,6,22,30,23,10,7
2,1610612751,Brooklyn Nets,13,2,11,0.154,624.0,493,1121,0.440,...,30,25,13,26,27,13,13,11,27,26
3,1610612766,Charlotte Hornets,13,4,9,0.308,629.0,532,1148,0.463,...,14,16,23,27,16,2,4,16,20,21
4,1610612741,Chicago Bulls,12,6,6,0.500,586.0,520,1080,0.481,...,18,13,9,23,18,22,5,28,26,19
5,1610612739,Cleveland Cavaliers,14,9,5,0.643,677.0,593,1308,0.453,...,8,5,17,5,2,16,27,2,3,11
6,1610612742,Dallas Mavericks,14,4,10,0.286,687.0,555,1228,0.452,...,2,17,28,9,1,19,18,3,16,24
7,1610612743,Denver Nuggets,12,10,2,0.833,581.0,537,1062,0.506,...,20,15,3,24,27,10,1,25,22,2
8,1610612765,Detroit Pistons,13,11,2,0.846,629.0,561,1188,0.472,...,7,17,12,7,4,14,21,9,19,6
9,1610612744,Golden State Warriors,15,9,6,0.600,725.0,590,1299,0.454,...,4,2,30,1,14,12,25,13,1,16


In [21]:
cols_to_scale = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'PTS']
team_stats[cols_to_scale] = team_stats[cols_to_scale].div(team_stats['GP'], axis=0)

In [22]:
matchups = [[16, 5], [11, 8], [12, 22], [19, 15], [3, 27], [6, 17], [20, 18], [4, 7]]

In [24]:
for i, matchup in enumerate(matchups):
  away = team_stats.iloc[[matchup[0]]]
  home = team_stats.iloc[[matchup[1]]]

  away = away.reset_index(drop=True)
  home = home.reset_index(drop=True)

  X_new1 = pd.concat([away, home.add_suffix('_OPP')], axis=1)
  X_new1 = X_new1.drop(columns=[c for c in X_new1.columns if c not in X_verify.columns])

  X_new2 = pd.concat([home, away.add_suffix('_OPP')], axis=1)
  X_new2 = X_new2.drop(columns=[c for c in X_new2.columns if c not in X_verify.columns])

  X_new = pd.concat([X_new1, X_new2]).reset_index(drop=True)

  print(clf.predict(X_new))
  probability = clf.predict_proba(X_new)

  print(f"{team_stats.iloc[[matchup[0]]]['TEAM_NAME']} win chance: {probability[0][1]*100}% / lose chance: {probability[0][0]*100}%")
  print(f"{team_stats.iloc[[matchup[1]]]['TEAM_NAME']} win chance: {probability[1][1]*100}% / lose chance: {probability[1][0]*100}%")

[1 1]
16    Milwaukee Bucks
Name: TEAM_NAME, dtype: object win chance: 50.67114093546737% / lose chance: 49.32885906453263%
5    Cleveland Cavaliers
Name: TEAM_NAME, dtype: object win chance: 50.67114093546737% / lose chance: 49.32885906453263%
[1 1]
11    Indiana Pacers
Name: TEAM_NAME, dtype: object win chance: 50.67114094192543% / lose chance: 49.32885905807457%
8    Detroit Pistons
Name: TEAM_NAME, dtype: object win chance: 50.67114094192543% / lose chance: 49.32885905807457%
[1 1]
12    LA Clippers
Name: TEAM_NAME, dtype: object win chance: 50.67114093817558% / lose chance: 49.32885906182442%
22    Philadelphia 76ers
Name: TEAM_NAME, dtype: object win chance: 50.67114093817558% / lose chance: 49.32885906182442%
[1 1]
19    New York Knicks
Name: TEAM_NAME, dtype: object win chance: 50.671140937967266% / lose chance: 49.328859062032734%
15    Miami Heat
Name: TEAM_NAME, dtype: object win chance: 50.671140937967266% / lose chance: 49.328859062032734%
[1 1]
3    Charlotte Hornets
Name